# Report
### This notebook is used to create visualizations and reports for the trading simulation.

## Step 1: Import Required Modules
- Import the necessary functions and modules for creating the report.


In [1]:
from create_report import *

## Step 2: Prepare Simulation Results
- Before running this notebook, make sure you have the results of the simulation prepared.
- Below, insert the path to the simulation results pickle files - one or more files.
- Ensure the files are in the correct chronological order.


In [2]:
file_paths = ['../../../data/2025-04-29_21-32-24-server_data.pickle', '../../../data/2025-04-29_21-33-11-server_data.pickle']
file_paths = ['../../../data/2025-04-24_06-36-32-server_data.pickle'] # exp2 no spoofing
users, timestamps, mid_prices = load_data(file_paths)

Processing file: ../../../data/2025-04-24_06-36-32-server_data.pickle


## Step 3: Generate Results Table
- Create a results table summarizing the performance of traders.
- The table will display key statistics and the top traders.


In [8]:
stats_df, top_n = create_results_table(users, mid_prices, censor=False, top_n=10)
display(stats_df)

Stock fee income: 2790.1966980003617


,Name,Final Balance,Return (%),Avg Volume Per Step,Max Volume,Avg Balance,Num Orders
1,ql_trader,10109.17,1.09,54.88,55,-5519.80,37
2,bayesian_trader,10034.05,0.34,11.19,23,-1130.97,18
3,linear_trader,10029.61,0.30,11.09,20,-1120.54,19
4,swing_trader,10013.61,0.14,3.09,9,-312.67,3
5,lstm_trader,10003.44,0.03,1.13,3,-114.02,27
6,range_trader,10000.00,0.00,0.00,0,0.00,0
7,momentum_trader_percentage_change,9998.82,-0.01,0.00,0,0.00,27
8,momentum_trader_SMA,9998.82,-0.01,0.00,0,0.00,27
9,momentum_trader_EMA,9998.82,-0.01,0.00,0,0.00,27
10,lasso_trader,9958.25,-0.42,59.84,89,-5987.86,2180


## Step 4: Visualize Top Traders
- Generate an interactive plot to visualize the performance of the top traders.


In [9]:
plot_best_traders_interactive(users, timestamps, mid_prices, top_n, censor=False)